In [1]:
import pyspark
import builtins as b
from datetime import datetime, timedelta, date
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
import sys, traceback

spark = SparkSession.builder.master("local[*]") \
                    .appName('tcc') \
                    .getOrCreate()

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
spark.conf.set("spark.sql.sources.partitionOverwriteMode","dynamic")

In [3]:
ciha = spark.read.parquet("bases finais/ciha/")

ciha = ciha.select(
    col("ANO_CMPT").alias("ano_cmpt"),
    col("MES_CMPT").alias("mes_cmpt"),
    col("CGC_HOSP").alias("cgc_hosp"),
    col("MUNIC_RES").alias("munic_res"),
    to_date("NASC", "yyyyMMdd").alias("nasc"),
    col("SEXO").alias("sexo"),
    col("UTI_MES_TO").alias("uti_mes_to"),
    col("UTI_INT_TO").alias("uti_int_to"),
    col("PROC_REA").alias("proc_rea"),
    col("QT_PROC").alias("qt_proc"),
    to_date("DT_ATEND", "yyyyMMdd").alias("dt_atend"),
    to_date("DT_SAIDA", "yyyyMMdd").alias("dt_saida"),
    col("DIAG_PRINC").alias("diag_princ"),
    col("DIAG_SECUN").alias("diag_secun"),
    col("COBRANCA").alias("cobranca"),
    col("NATUREZA").alias("natureza"),
    col("GESTAO").alias("gestao"),
    col("MUNIC_MOV").alias("munic_mov"),
    col("COD_IDADE").alias("cod_idade"),
    col("IDADE").alias("idade"),
    col("DIAS_PERM").alias("dias_perm"),
    col("MORTE").alias("morte"),
    col("CNES").alias("cnes"),
    col("FONTE").alias("fonte"),
    col("MODALIDADE").alias("modalidade")
)

ciha.printSchema()

# print(ciha.count())

ciha.show()

root
 |-- ano_cmpt: string (nullable = true)
 |-- mes_cmpt: string (nullable = true)
 |-- cgc_hosp: string (nullable = true)
 |-- munic_res: string (nullable = true)
 |-- nasc: date (nullable = true)
 |-- sexo: string (nullable = true)
 |-- uti_mes_to: string (nullable = true)
 |-- uti_int_to: string (nullable = true)
 |-- proc_rea: string (nullable = true)
 |-- qt_proc: string (nullable = true)
 |-- dt_atend: date (nullable = true)
 |-- dt_saida: date (nullable = true)
 |-- diag_princ: string (nullable = true)
 |-- diag_secun: string (nullable = true)
 |-- cobranca: string (nullable = true)
 |-- natureza: string (nullable = true)
 |-- gestao: string (nullable = true)
 |-- munic_mov: string (nullable = true)
 |-- cod_idade: string (nullable = true)
 |-- idade: string (nullable = true)
 |-- dias_perm: string (nullable = true)
 |-- morte: string (nullable = true)
 |-- cnes: string (nullable = true)
 |-- fonte: string (nullable = true)
 |-- modalidade: string (nullable = true)

+--------+

In [5]:
ciha.select("ano_cmpt").groupBy("ano_cmpt").count().orderBy("ano_cmpt").show()

+--------+--------+
|ano_cmpt|   count|
+--------+--------+
|    2011|13455228|
|    2012| 7493554|
|    2013|13539741|
|    2014| 9157732|
|    2015| 8976001|
|    2016|10116122|
|    2017|11664889|
|    2018|11277506|
|    2019|11150677|
|    2020| 8660219|
|    2021| 9908547|
|    2022|10433583|
|    2023| 1438347|
+--------+--------+



In [6]:
ciha.filter("ano_cmpt between 2011 and 2022").count()

125833799

In [8]:
municipios = spark.read.option("header", True).option("sep", ";").option("encoding", "ISO-8859-1").csv("bases finais/municipios/municipios.csv")

municipios = municipios.select(
    col("UF").alias("uf"),
    col("Nome_UF").alias("nome_uf"),
    col("Município").alias("municipio"),
    col("Código Município Completo").alias("cod_municipio_completo"),
    col("Nome_Município").alias("nome_municipio"),
    substring('Código Município Completo', 1,6).alias("cod_municipio")
)

municipios.printSchema()

print(municipios.count())

municipios.show()

root
 |-- uf: string (nullable = true)
 |-- nome_uf: string (nullable = true)
 |-- municipio: string (nullable = true)
 |-- cod_municipio_completo: string (nullable = true)
 |-- nome_municipio: string (nullable = true)
 |-- cod_municipio: string (nullable = true)

5570
+---+--------+---------+----------------------+--------------------+-------------+
| uf| nome_uf|municipio|cod_municipio_completo|      nome_municipio|cod_municipio|
+---+--------+---------+----------------------+--------------------+-------------+
| 11|Rondônia|    00015|               1100015|Alta Floresta D'O...|       110001|
| 11|Rondônia|    00379|               1100379|Alto Alegre dos P...|       110037|
| 11|Rondônia|    00403|               1100403|        Alto Paraíso|       110040|
| 11|Rondônia|    00346|               1100346|    Alvorada D'Oeste|       110034|
| 11|Rondônia|    00023|               1100023|           Ariquemes|       110002|
| 11|Rondônia|    00452|               1100452|             Buriti

In [9]:
estados = spark.read.option("header", True).option("sep", ";").option("encoding", "ISO-8859-1").csv("bases finais/municipios/estados.csv")

estados = estados.select(
    col("COD_UF").alias("cod_uf"),
    col("NOME_UF").alias("nome_uf"),
    col("REGIAO").alias("regiao"),
    col("SIGLA").alias("sigla")
)

estados.printSchema()

print(estados.count())

estados.show()

root
 |-- cod_uf: string (nullable = true)
 |-- nome_uf: string (nullable = true)
 |-- regiao: string (nullable = true)
 |-- sigla: string (nullable = true)

27
+------+-------------------+--------+-----+
|cod_uf|            nome_uf|  regiao|sigla|
+------+-------------------+--------+-----+
|    11|           Rondônia|   Norte|   RO|
|    12|               Acre|   Norte|   AC|
|    13|           Amazonas|   Norte|   AM|
|    14|            Roraima|   Norte|   RR|
|    15|               Pará|   Norte|   PA|
|    16|              Amapá|   Norte|   AP|
|    17|          Tocantins|   Norte|   TO|
|    21|           Maranhão|Nordeste|   MA|
|    22|              Piauí|Nordeste|   PI|
|    23|              Ceará|Nordeste|   CE|
|    24|Rio Grande do Norte|Nordeste|   RN|
|    25|            Paraíba|Nordeste|   PB|
|    26|         Pernambuco|Nordeste|   PE|
|    27|            Alagoas|Nordeste|   AL|
|    28|            Sergipe|Nordeste|   SE|
|    29|              Bahia|Nordeste|   BA|
|  

In [10]:
idhm = spark.read.option("header", True).option("sep", ",").csv("bases finais/ibge/idhm_2010.csv")

idhm = idhm.select(
    col("COD_MUNICIPIO").alias("cod_municipio"),
    col("IDHM_2010").cast("float").alias("idhm_2010")
)

idhm.printSchema()

print(idhm.count())

idhm.show()

root
 |-- cod_municipio: string (nullable = true)
 |-- idhm_2010: float (nullable = true)

5564
+-------------+---------+
|cod_municipio|idhm_2010|
+-------------+---------+
|       110001|    0.641|
|       110002|    0.702|
|       110003|     0.65|
|       110004|    0.718|
|       110005|    0.692|
|       110006|    0.685|
|       110007|    0.613|
|       110008|    0.611|
|       110009|    0.672|
|       110010|    0.657|
|       110011|    0.689|
|       110012|    0.714|
|       110013|    0.596|
|       110014|    0.643|
|       110015|    0.682|
|       110018|     0.71|
|       110020|    0.736|
|       110025|    0.664|
|       110026|    0.643|
|       110028|      0.7|
+-------------+---------+
only showing top 20 rows



In [10]:
ibge = spark.read.option("header", True).option("sep", ",").csv("bases finais/ibge/ibge.csv")

ibge = ibge.select(
    col("COD_MUNICIPIO").alias("cod_municipio"),
    col("POPULACAO_RESIDENTE").cast("integer").alias("populacao_residente"),
    col("AREA_UNIDADE_TERRITORIAL").cast("float").alias("area_unidade_territorial")
)

ibge.printSchema()

print(ibge.count())

ibge.show()

root
 |-- cod_municipio: string (nullable = true)
 |-- populacao_residente: integer (nullable = true)
 |-- area_unidade_territorial: float (nullable = true)

5564
+-------------+-------------------+------------------------+
|cod_municipio|populacao_residente|area_unidade_territorial|
+-------------+-------------------+------------------------+
|       110001|              24392|                7067.025|
|       110002|              90353|                4426.571|
|       110003|               6313|                1314.362|
|       110004|              78574|                3792.801|
|       110005|              17029|                  2783.3|
|       110006|              18591|                1451.061|
|       110007|               8783|                3060.321|
|       110008|              13678|                4987.177|
|       110009|              28729|                4518.034|
|       110010|              41656|               24855.725|
|       110011|              52005|         

In [7]:
feriados = spark.read.option("header", True).option("sep", ";").option("encoding", "ISO-8859-1").csv("bases finais/feriados/")

feriados.printSchema()

print(feriados.count())

feriados.show()

root
 |-- data: string (nullable = true)
 |-- nome: string (nullable = true)
 |-- tipo: string (nullable = true)
 |-- descricao: string (nullable = true)
 |-- uf: string (nullable = true)
 |-- municipio: string (nullable = true)
 |-- cod_municipio: string (nullable = true)

154378
+----------+--------------------+--------+--------------------+---+---------+-------------+
|      data|                nome|    tipo|           descricao| uf|municipio|cod_municipio|
+----------+--------------------+--------+--------------------+---+---------+-------------+
|2015-01-20|     Dia do Católico|ESTADUAL|Dia 20 de janeiro...| AC|     null|         null|
|2015-01-23|   Dia do Evangélico|ESTADUAL|Dia 23 de janeiro...| AC|     null|         null|
|2015-03-08|       Dia da Mulher|ESTADUAL|Feriado conforme ...| AC|     null|         null|
|2015-06-15| Aniversário do Acre|ESTADUAL|De acordo com a L...| AC|     null|         null|
|2015-08-06|   Revolução Acreana|ESTADUAL|                null| AC|     nu

In [8]:
feriados_prep = feriados.withColumn("facultativo", when(col("tipo") == lit("FACULTATIVO"), lit(True)).otherwise(lit(False))).groupBy(
    col("data")
).agg(
    collect_list(
        struct(
            "data",
            "nome",
            "tipo",
            "descricao",
            "uf",
            "municipio",
            "cod_municipio"
        )
    ).alias("itens")
).rdd.map(lambda x: x.asDict()).collect()

feriados_prep = {
    x["data"]: [y.asDict() for y in x["itens"]]
    for x in feriados_prep
}

sorted_keys = list(feriados_prep.keys())
sorted_keys.sort()

feriados_prep = {
    k: feriados_prep[k]
    for k in sorted_keys
}

In [9]:
def checa_feriados(data, cod_municipio=None, uf_estado=None):
    feriados_possiveis = []
    erro = ""

    try:
        if data in feriados_prep.keys():
            for feriado in feriados_prep[data]:
                if feriado["tipo"] in ["NACIONAL", "FACULTATIVO"]:
                    feriados_possiveis.append(feriado)
                elif feriado["tipo"] in ["ESTADUAL"]:
                    if uf_estado is not None and feriado["uf"] == uf_estado:
                        feriados_possiveis.append(feriado)
                elif feriado["tipo"] in ["MUNICIPAL"]:
                    if cod_municipio is not None and feriado["cod_municipio"] == cod_municipio:
                        feriados_possiveis.append(feriado)
            
            feriado = True
            dist_feriado = 0
            
        if len(feriados_possiveis) == 0:
            feriado_info = None
            feriado = False
            
            mais_prox = ""
            dist_prox = 999
            for data_feriado in feriados_prep:
                data_obj = datetime.strptime(str(data), "%Y-%m-%d").date()
                feriado_obj = datetime.strptime(data_feriado, "%Y-%m-%d").date()
                d = b.abs((data_obj - feriado_obj).days)
                
                
                if d < dist_prox:
                    checked = False
                    for check_feriado in feriados_prep[data_feriado]:
                        if ((check_feriado["tipo"] in ["NACIONAL", "FACULTATIVO"])
                            or (check_feriado["tipo"] == "ESTADUAL" and check_feriado["uf"] == uf_estado)
                            or (check_feriado["tipo"] == "MUNICIPAL" and check_feriado["cod_municipio"] == cod_municipio)):
                            checked = True
                    if checked:
                        dist_prox = d
                        mais_prox = feriado
                        feriado_info = list(b.filter(None, [
                            x if ((x["tipo"] in ["NACIONAL", "FACULTATIVO"])
                                or (x["tipo"] == "ESTADUAL" and x["uf"] == uf_estado)
                                or (x["tipo"] == "MUNICIPAL" and x["cod_municipio"] == cod_municipio))
                            else None
                            for x in feriados_prep[data_feriado]
                        ]))
            
            dist_feriado = dist_prox 
            feriados_possiveis += feriado_info
            
    except Exception as e:
        ex_type, ex, tb = sys.exc_info()
        feriado = None
        dist_feriado = None
        feriados_possiveis = None
        erro = str(traceback.extract_tb(tb)) + " / " + str(e)

    return {
        "feriado": feriado,
        "distancia_feriado": dist_feriado,
        "feriado_info": feriados_possiveis,
        "erro": erro
    }

udf_checa_feriados = udf(checa_feriados, StructType([
    StructField("feriado", StringType(), True),
    StructField("distancia_feriado", IntegerType(), True),
    StructField("feriado_info", ArrayType(StructType([
        StructField("data", StringType(), True),
        StructField("nome", StringType(), True),
        StructField("tipo", StringType(), True),
        StructField("descricao", StringType(), True),
        StructField("uf", StringType(), True),
        StructField("municipio", StringType(), True),
        StructField("cod_municipio", StringType(), True)
    ])), True),
    StructField("erro", StringType(), True)
]))

In [12]:
prefinal = (ciha.alias("ciha")
            .join(municipios.alias("mun"), col("ciha.munic_res") == col("mun.cod_municipio"), "left")
            .join(estados.alias("est"), col("mun.uf") == col("est.cod_uf"), "left")
            .join(idhm.alias("idhm"), col("ciha.munic_res") == col("idhm.cod_municipio"), "left")
            .join(ibge.alias("ibge"), col("ciha.munic_res") == col("ibge.cod_municipio"), "left")

            .select(
                col("ciha.*"),
                col("mun.nome_uf"),
                col("mun.nome_municipio"),
                col("est.sigla"),
                col("est.regiao"),
                col("idhm.idhm_2010").alias("idhm"),
                col("ibge.populacao_residente"),
                col("ibge.area_unidade_territorial")
            )
           
           )

prefinal.show()

+--------+--------+--------------+---------+----------+----+----------+----------+----------+-------+----------+----------+----------+----------+--------+--------+------+---------+---------+-----+---------+-----+-------+-----+----------+---------+-------------------+-----+-------+-----+-------------------+------------------------+
|ano_cmpt|mes_cmpt|      cgc_hosp|munic_res|      nasc|sexo|uti_mes_to|uti_int_to|  proc_rea|qt_proc|  dt_atend|  dt_saida|diag_princ|diag_secun|cobranca|natureza|gestao|munic_mov|cod_idade|idade|dias_perm|morte|   cnes|fonte|modalidade|  nome_uf|     nome_municipio|sigla| regiao| idhm|populacao_residente|area_unidade_territorial|
+--------+--------+--------------+---------+----------+----+----------+----------+----------+-------+----------+----------+----------+----------+--------+--------+------+---------+---------+-----+---------+-----+-------+-----+----------+---------+-------------------+-----+-------+-----+-------------------+------------------------+
|

In [16]:
prefinal.select("dt_atend", "sigla", "munic_res").count()

127272146

In [17]:
prefinal.select("dt_atend", "munic_res", "sigla").distinct().count()

1694708

In [23]:
preferiado = prefinal.select("dt_atend", "munic_res", "sigla").distinct()
preferiado.write.mode("overwrite").parquet("bases temp/preferiado/")

In [13]:
preferiado_partitionated = spark.read.parquet("bases temp/preferiado/")
preferiado_partitionated.write.partitionBy("sigla").mode("overwrite").parquet("bases temp/preferiado_partitionated/")

In [15]:
preferiado_partitionated.groupBy("sigla").count().show()

+-----+------+
|sigla| count|
+-----+------+
|   SC| 84543|
|   RO|  5791|
|   PI|  8739|
|   GO| 29475|
| null| 86015|
|   TO|   706|
|   MT| 28529|
|   SP|608758|
|   ES| 45812|
|   PB| 11458|
|   RS|171508|
|   MS| 30015|
|   AL| 19926|
|   MG|259669|
|   PA| 34438|
|   BA| 31469|
|   SE|  8882|
|   PE| 22223|
|   CE| 48754|
|   RN|  8007|
+-----+------+
only showing top 20 rows



In [32]:
# withferiado = spark.read.parquet("bases temp/preferiado_partitionated/").filter(f"sigla is null").withColumn("feriado", udf_checa_feriados(col("dt_atend"), col("munic_res"), col("sigla")))
# withferiado.write.partitionBy("sigla").mode("overwrite").parquet("bases temp/withferiado/")

In [13]:
feriado_done = spark.read.parquet("bases temp/withferiado/")
feriado_done.show(truncate = False)

+----------+---------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+
|dt_atend  |munic_res|feriado                                                                                                                                                                                                                                                                                                                                                                                                             |sigla|
+----------+---------+--------------------------------------------------------------------------------------------------------------

In [12]:
feriado_done.printSchema()

root
 |-- dt_atend: date (nullable = true)
 |-- munic_res: string (nullable = true)
 |-- feriado: struct (nullable = true)
 |    |-- feriado: string (nullable = true)
 |    |-- distancia_feriado: integer (nullable = true)
 |    |-- feriado_info: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- data: string (nullable = true)
 |    |    |    |-- nome: string (nullable = true)
 |    |    |    |-- tipo: string (nullable = true)
 |    |    |    |-- descricao: string (nullable = true)
 |    |    |    |-- uf: string (nullable = true)
 |    |    |    |-- municipio: string (nullable = true)
 |    |    |    |-- cod_municipio: string (nullable = true)
 |    |-- erro: string (nullable = true)
 |-- sigla: string (nullable = true)



In [16]:
feriado_done.count()

1544717

In [14]:
cid10 = spark.read.option("header", True).option("sep", ";").option("encoding", "ISO-8859-1").csv("bases finais/cid10/CID-10-SUBCATEGORIAS.CSV")

cid10 = cid10.select(
    col("SUBCAT").alias("sub_cat"),
    col("CLASSIF").alias("classificacao"),
    col("RESTRSEXO").alias("restr_sexo"),
    col("CAUSAOBITO").alias("causa_obito"),
    col("DESCRICAO").alias("descricao"),
    col("DESCRABREV").alias("desc_abrev"),
    col("REFER").alias("refer"),
    col("EXCLUIDOS").alias("excluidos")
)

In [17]:
cid10.show(truncate=False)

+-------+-------------+----------+-----------+----------------------------------------------------+-------------------------------------------------+-----+---------+
|sub_cat|classificacao|restr_sexo|causa_obito|descricao                                           |desc_abrev                                       |refer|excluidos|
+-------+-------------+----------+-----------+----------------------------------------------------+-------------------------------------------------+-----+---------+
|A000   |null         |null      |null       |Cólera devida a Vibrio cholerae 01, biótipo cholerae|A00.0 Colera dev Vibrio cholerae 01 biot cholerae|null |null     |
|A001   |null         |null      |null       |Cólera devida a Vibrio cholerae 01, biótipo El Tor  |A00.1 Colera dev Vibrio cholerae 01 biot El Tor  |null |null     |
|A009   |null         |null      |null       |Cólera não especificada                             |A00.9 Colera NE                                  |null |null     |
|A01

In [20]:
prefinal.count()

127272146

In [15]:
final = (prefinal.alias("pf")
    .join(feriado_done.alias("fd"), 
          (col("pf.dt_atend") == col("fd.dt_atend"))
          & (col("pf.munic_res") == col("fd.munic_res"))
          & (col("pf.sigla") == col("fd.sigla")),
          "left")
    .join(cid10.alias("cid10_princ"),
          col("pf.diag_princ") == col("cid10_princ.sub_cat"),
         "left")
    .join(cid10.alias("cid10_secun"),
          col("pf.diag_secun") == col("cid10_secun.sub_cat"),
         "left")
    .filter("pf.nome_municipio is not null")
    .filter("fd.feriado.feriado is not null")
    .select(
        col("pf.ano_cmpt"),
        col("pf.mes_cmpt"),
        col("pf.cgc_hosp"),
        col("pf.munic_res"),
        col("pf.nasc"),
        col("pf.sexo"),
        col("pf.uti_mes_to"),
        col("pf.uti_int_to"),
        col("pf.proc_rea"),
        col("pf.qt_proc"),
        col("pf.dt_atend"),
        col("pf.dt_saida"),
        col("pf.diag_princ"),
        col("pf.diag_secun"),
        col("pf.cobranca"),
        col("pf.natureza"),
        col("pf.gestao"),
        col("pf.munic_mov"),
        col("pf.cod_idade"),
        col("pf.idade"),
        col("pf.dias_perm"),
        col("pf.morte"),
        col("pf.cnes"),
        col("pf.fonte"),
        col("pf.modalidade"),
        col("pf.nome_uf"),
        col("pf.nome_municipio"),
        col("pf.sigla"),
        col("pf.regiao"),
        col("pf.idhm"),
        col("pf.populacao_residente"),
        col("pf.area_unidade_territorial"),
        col("cid10_princ.descricao").alias("diag_princ_desc"),
        col("cid10_secun.descricao").alias("diag_secun_desc"),
        struct(col("cid10_princ.*")).alias("diag_princ_detalhes"),
        struct(col("cid10_secun.*")).alias("diag_secun_detalhes"),
        col("fd.feriado.feriado").alias("feriado"),
        col("fd.feriado.distancia_feriado").alias("distancia_feriado"),
        col("fd.feriado.feriado_info").alias("feriado_info")
    )        
)

In [27]:
final.limit(20).show()

+--------+--------+--------------+---------+----------+----+----------+----------+----------+-------+----------+----------+----------+----------+--------+--------+------+---------+---------+-----+---------+-----+-------+-----+----------+------------------+--------------------+-----+------------+-----+--------------------+--------------------+--------------------+--------------------+-------+-----------------+--------------------+
|ano_cmpt|mes_cmpt|      cgc_hosp|munic_res|      nasc|sexo|uti_mes_to|uti_int_to|  proc_rea|qt_proc|  dt_atend|  dt_saida|diag_princ|diag_secun|cobranca|natureza|gestao|munic_mov|cod_idade|idade|dias_perm|morte|   cnes|fonte|modalidade|           nome_uf|      nome_municipio|sigla|      regiao| idhm|     diag_princ_desc|     diag_secun_desc| diag_princ_detalhes| diag_secun_detalhes|feriado|distancia_feriado|        feriado_info|
+--------+--------+--------------+---------+----------+----+----------+----------+----------+-------+----------+----------+---------

In [41]:
final.count()

99162294

In [42]:
final.filter("feriado is null").count()

0

In [16]:
final.write.mode("overwrite").partitionBy("sigla").parquet("bases finais/final/")

In [7]:
final = spark.read.parquet("bases finais/final/")

In [8]:
final.filter("ano_cmpt between 2011 and 2022").count()

98231863

In [18]:
final_sample = (final
     .withColumn("diag_princ_detalhes", to_json("diag_princ_detalhes"))
     .withColumn("diag_secun_detalhes", to_json("diag_secun_detalhes"))
     .withColumn("feriado_info", to_json("feriado_info"))
     .sample(fraction=0.001, seed = 1010)
)

(final_sample.repartition(1)
     .write
     .mode("overwrite")
     .option("header", True)
     .option("sep", ";")
     .csv("bases finais/final_sample/")
)

In [24]:
final_sample.columns

['ano_cmpt',
 'mes_cmpt',
 'cgc_hosp',
 'munic_res',
 'nasc',
 'sexo',
 'uti_mes_to',
 'uti_int_to',
 'proc_rea',
 'qt_proc',
 'dt_atend',
 'dt_saida',
 'diag_princ',
 'diag_secun',
 'cobranca',
 'natureza',
 'gestao',
 'munic_mov',
 'cod_idade',
 'idade',
 'dias_perm',
 'morte',
 'cnes',
 'fonte',
 'modalidade',
 'nome_uf',
 'nome_municipio',
 'sigla',
 'regiao',
 'idhm',
 'diag_princ_desc',
 'diag_secun_desc',
 'diag_princ_detalhes',
 'diag_secun_detalhes',
 'feriado',
 'distancia_feriado',
 'feriado_info']

In [15]:
final.filter("diag_princ like '%N47%' and morte = 1").count()

536

In [ ]:
# To do
#   Add mesoregiao
#   Add dia da semana
#   